In [67]:
import pandas as pd
import random
from sqlalchemy import create_engine

1. Charger les fichiers et générer des IDs uniques

In [68]:
# Charger les fichiers sources
file1 = 'data\\ObesityDataSet_raw_and_data_sinthetic.csv'
file2 = 'data\\lifestyle_sustainability_data.csv'
file3 = 'data\\Sleep_health_and_lifestyle_dataset.csv'
file4 = 'data\\DietSurvey_toJson.json'

# Charger les fichiers dans des DataFrames
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)
df3 = pd.read_csv(file3)
df4 = pd.read_json(file4)

# Ajouter des IDs uniques pour chaque fichier
df1['id_person'] = range(1, len(df1) + 1)
df2['id_person'] = range(len(df1) + 1, len(df1) + len(df2) + 1)
df3['id_person'] = range(len(df1) + len(df2) + 1, len(df1) + len(df2) + len(df3) + 1)
df4['id_person'] = range(len(df1) + len(df2) + len(df3) + 1, len(df1) + len(df2) + len(df3) + len(df4) + 1)

# Afficher un aperçu des IDs pour validation
print("Aperçu des IDs générés pour chaque fichier :")
print(df1[['id_person']].head())
print(df2[['id_person']].head())
print(df3[['id_person']].head())
print(df4[['id_person']].head())


Aperçu des IDs générés pour chaque fichier :
   id_person
0          1
1          2
2          3
3          4
4          5
   id_person
0       2112
1       2113
2       2114
3       2115
4       2116
   id_person
0       2611
1       2612
2       2613
3       2614
4       2615
   id_person
0       2985
1       2986
2       2987
3       2988
4       2989


Étape 2 : Créer la table principale Persons avec des données spécifiques

On génère un age contenu dans la fourchette de la colonne age (ex : "Age": "18-24") pour les données qui viennent du fichier Json.

In [69]:
# Fonction pour générer un âge aléatoire basé sur une fourchette
def generate_random_age(age_range):
    if age_range == "Under 18":
        return random.randint(15, 17)
    elif age_range == "18-24":
        return random.randint(18, 24)
    elif age_range == "25-34":
        return random.randint(25, 34)
    elif age_range == "35-44":
        return random.randint(35, 44)
    elif age_range == "45-54":
        return random.randint(45, 54)
    elif age_range == "55-64":
        return random.randint(55, 64)
    elif age_range == "Above 65":
        return random.randint(66, 100)  
    else:
        return None  # Gère les cas où la fourchette est manquante ou incorrecte


On associe la catégorie d'age correspondante.

In [70]:
# Fonction pour obtenir l'ID de la catégorie d'âge en fonction de la tranche
def get_age_category_id(age):
    if 18 <= age <= 24:
        return 1  # ID pour 18-24
    elif 25 <= age <= 34:
        return 6  # ID pour 25-34
    elif 35 <= age <= 44:
        return 5  # ID pour 35-44
    elif 45 <= age <= 54:
        return 2  # ID pour 45-54
    elif 55 <= age <= 64:
        return 7  # ID pour 55-64
    elif age >= 65:
        return 3  # ID pour Above 65
    else:
        return 4  # ID pour Under 18

In [71]:
# persons1 - df1
persons1 = df1[['id_person', 'Gender', 'Age', 'family_history_with_overweight', 'Height', 'Weight']].rename(
    columns={'Gender': 'gender', 'Age': 'age', 'family_history_with_overweight': 'health_history', 'Height': 'height', 'Weight': 'weight'}
)
print("persons1 préparé.")

# persons2 - df2
persons2 = df2[['id_person', 'Gender', 'Age']].rename(columns={'Gender': 'gender', 'Age': 'age'})
persons2['health_history'] = None
persons2['height'] = None
persons2['weight'] = None
print("persons2 préparé.")

# persons3 - df3
persons3 = df3[['id_person', 'Gender', 'Age', 'BMI Category']].rename(
    columns={'Gender': 'gender', 'Age': 'age', 'BMI Category': 'bmi_category'}
)

persons3['health_history'] = persons3['bmi_category'].apply(lambda x: 'yes' if x == 'Overweight' else 'no')
persons3.drop(columns=['bmi_category'], inplace=True)

# Ajouter 'height' et 'weight' comme None car ces informations ne sont pas présentes dans df3
persons3['height'] = None
persons3['weight'] = None

# persons4 - df4
persons4 = df4[['id_person', 'Gender', 'Age']].rename(columns={'Gender': 'gender', 'Age': 'age'})
persons4['health_history'] = None
persons4['height'] = None
persons4['weight'] = None

# Si Age est une fourchette, générer un âge aléatoire pour chaque ligne de `persons4`
persons4['generated_age'] = persons4['age'].apply(generate_random_age)
persons4['age'] = persons4['generated_age']  # Remplacer l'ancienne colonne 'age' par l'âge généré
print("persons4 préparé avec âges générés.")

# Supprimer la colonne 'generated_age' pour ne pas la laisser dans le DataFrame final
persons4.drop(columns=['generated_age'], inplace=True)
print("persons4 préparé avec âges générés et colonne 'generated_age' supprimée.")

# Combinaison des données
persons = pd.concat([persons1, persons2, persons3, persons4], ignore_index=True)
print("Données combinées dans `persons`.")

# Associer l'ID de la catégorie d'âge à chaque individu
persons['id_age_category'] = persons['age'].apply(get_age_category_id)

# Afficher un aperçu de la table finale
print("Table `Persons` avec `id_age_category` ajoutée :")
print(persons.head())

persons1 préparé.
persons2 préparé.
persons4 préparé avec âges générés.
persons4 préparé avec âges générés et colonne 'generated_age' supprimée.
Données combinées dans `persons`.
Table `Persons` avec `id_age_category` ajoutée :
   id_person  gender   age health_history  height  weight  id_age_category
0          1  Female  21.0            yes    1.62    64.0                1
1          2  Female  21.0            yes    1.52    56.0                1
2          3    Male  23.0            yes    1.80    77.0                1
3          4    Male  27.0             no    1.80    87.0                6
4          5    Male  22.0             no    1.78    89.8                1


C:\Users\user\AppData\Local\Temp\ipykernel_14100\446324303.py:42: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  persons = pd.concat([persons1, persons2, persons3, persons4], ignore_index=True)


In [72]:
# Remplacer "Prefer not to say" par "Other" dans la colonne 'gender'
persons['gender'] = persons['gender'].replace('Prefer not to say', 'Other')

# Vérifier les changements
print(persons['gender'].value_counts())


gender
Male          1473
Female        1433
Non-Binary      81
Other           75
Name: count, dtype: int64


In [73]:
# Afficher un aperçu des données combinées
print("Aperçu de la table `persons` :")
print(persons.head())

Aperçu de la table `persons` :
   id_person  gender   age health_history  height  weight  id_age_category
0          1  Female  21.0            yes    1.62    64.0                1
1          2  Female  21.0            yes    1.52    56.0                1
2          3    Male  23.0            yes    1.80    77.0                1
3          4    Male  27.0             no    1.80    87.0                6
4          5    Male  22.0             no    1.78    89.8                1


In [1]:
from db_utils import get_db_engine

# Obtenir le moteur de base de données
engine = get_db_engine()

In [ ]:
# Sélectionner les données à insérer
persons_to_insert = persons[['id_person', 'gender', 'age', 'health_history', 'height', 'weight', 'id_age_category']]

# Insérer les données dans la table 'Persons'
persons_to_insert.to_sql('Persons', engine, if_exists='append', index=False)

# Confirmer que les données ont été insérées
print("Les données ont été insérées avec succès dans la table 'Persons'.")

Connexion à la base de données réussie !
Les données ont été insérées avec succès dans la table 'Persons'.


C:\Users\user\AppData\Local\Temp\ipykernel_14100\600092092.py:22: UserWarning: The provided table name 'Persons' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  persons_to_insert.to_sql('Persons', engine, if_exists='append', index=False)
